# Importing a Qiskit circuit into TQEC

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
from pathlib import Path

ASSETS_FOLDER = Path("../../assets/").resolve()

# To display interactive 3D visualisations, uncomment the following line.
# %matplotlib widget

This notebook illustrates the process of importing into TQEC circuits originally created with Qiskit. We'll be using a previously-designed 16-qubit GHZ, but the procedure is the same for any circuit saved as `.qpy`.

The notebook assumes:
- you have designed a Qiskit circuit and saved it to `.qpy` format, 
- you installed all dependency groups when setting up TQEC (see instructions [here](https://tqec.github.io/tqec/contributor_guide.html)).

## Load Qiskit circuit

The first step is, naturally, to load and inspect the Qiskit circuit.

Note that the circuit needs to be designed in a way that allows import into PyZX. This may or may not require you to adapt your circuit design practices. It really does depend on whether you already design with PyZX in mind or not. 

In [ ]:
from qiskit import qasm2, qpy

# NAME OF CIRCUIT
circuit_name = "ghz16"

# LOAD CIRCUIT
path_to_c = ASSETS_FOLDER / f"{circuit_name}_qiskit.qpy"
with open(path_to_c, "rb") as f:
    qc = qpy.load(f)[0]

print(f"\n=> Imported Qiskit circuit of name {circuit_name.upper()}:\n")
qc.draw(output="text", fold=200)

To load the circuit into PyZX, we need to export it to QASM.

In [ ]:
# CONVERT to QASM
qasm_str = qasm2.dumps(qc)

print(f"\n=> Converted Qiskit circuit of name {circuit_name.capitalize()} into the following QASM string:\n")
print(qasm_str)

## QASM to PyZX

Let's now go ahead and load the circuit into QASM.

In [ ]:
if qasm_str:
    import pyzx as zx
    zx.settings.colors = zx.rgb_colors

    # Import QASM into PyZX
    zx_circuit = zx.Circuit.from_qasm(qasm_str)

    # Convert incoming circuit into graph
    zx_graph = zx_circuit.to_graph()

    # Draw graph of incoming circuit
    zx.draw(zx_graph, labels = True)

The circuit is not optimised. It is possible to turn an unoptimised circuit into a TQEC `block_graph`, but the resulting `block_graph` would be quite large. 

So, it is better to optimise the circuit before converting it into a `block_graph`. 

For this, since PyZX's QASM compatibility is still in-development, we must first initialise the qubits and add any necessary measurements.

In [ ]:
if zx_graph:
    # States
    num_apply_state = zx_graph.num_inputs()
    zx_graph.apply_state('0' * num_apply_state)

    # Post-selection
    # Note! No post-selection is needed for this specific circuit.
    # The following line is included to clarify where and how one would undertake the operation
    # To apply post-selection, you would exchange the "/" for a "0" if you want the n-qubit post-selected (string is ordered by qubit).
    zx_graph.apply_effect('////////////////')

    # Draw graph of incoming circuit
    zx.draw(zx_graph, labels = True)

It is now possible to use standard PyZX methods to reduce the circuit/graph to a more optimal version of itself.

In [ ]:
random.seed(12)
if zx_graph:

    # Reduce circuit
    zx.full_reduce(zx_graph)

    # Draw reduced circuit
    zx.draw(zx_graph, labels = True, auto_layout=True)

## Algorithmic lattice surgery

It is now time to undertake lattice surgery on the PyZX graph.

The first step is to convert the graph into TQEC/topologiq's native graph object, a `simple_graph`.

The initial `simple_graph` graph is identical to the PyZX graph in terms of connectivity. It's just simpler and, by extension, easier to manipulate algorithmically. Also, even if connectivity remains the same, the visualisation of a `simple_graph` might look different than a PyZX figure because PyZX defines visualisation layouts using data that is not carried over into the `simple_graph`. 

In [ ]:
if zx_graph:

    from topologiq.utils.interop_pyzx import pyzx_g_to_simple_g
    from topologiq.utils.simple_grapher import simple_graph_vis

    # Transform ZX-graph into topologiq's native `simple_graph`
    simple_graph = pyzx_g_to_simple_g(zx_graph)

    # Print `simple_graph for inspection`
    # We extract `fig_data` from the `simple_graph` due to having been unable to use PyZX visualisations for this.
    # This does mean the overlay in future visualisations will not look exactly as the PyZX graph above.

    fig_data = simple_graph_vis(simple_graph)
    for k, v in simple_graph.items():
        print(f"{k}: {v}")

After, we can run the `simple_graph` by Topologiq.

It is worth noting before running Topologiq that the exact shape of the `simple_graph` might change significantly during the process. For example, surface code-based lattice surgery requires graphs where all spiders have at most four (4) legs/neighbours, which is evidently not the case of the graph drawn above. Topologiq has internal obfuscation operations to handle this kind of situations.

In [ ]:
if zx_graph:

    from topologiq.scripts.runner import runner

    # Parameters & hyper-parameters
    full_circuit_name = f"qiskit_{circuit_name}"
    vis = "final"  # Calls 3D visualisation at the end. `None` to deactivate.
    anim = None  # Best to avoid in a public notebook. Animation support depends a lot on the machine and rights.

    VALUE_FUNCTION_HYPERPARAMS = (
        -1,  # Weight for length of path
        -1,  # Weight for number of "beams" broken by path
    )

    kwargs = {
        "weights": VALUE_FUNCTION_HYPERPARAMS,
        "length_of_beams": 9,
    }

    # Run topologiq
    _, _, lattice_nodes, lattice_edges = runner(
        simple_graph,  # The simple_graph to be processed by Topologiq
        full_circuit_name,  # Name of the circuit
        min_succ_rate = 80,  # Runtime saving parameter (min % of total possible paths per edge)
        strip_ports = False,  # Remove open boundaries from an incoming graph
        hide_ports = False,  # Leave open boundaries in graph object but hide in visualisations
        max_attempts = 10,  # Maximum # of attempts to find a successful solution
        stop_on_first_success = True,  # Exit when any attempt is successful (False useful for automating stats)
        visualise = (vis, anim),  # (Visualisation mode, Animation mode)
        log_stats = False,  # Automatically log stats for all runs (requires writing privileges)
        debug = False,  # Enter debug mode (additional detail in visualisation)
        fig_data = None,  # Matplotlib object containing input ZX graph (to overlay over visualisations)
        **kwargs,  # {Weights for value function, Length of beams}
    )

And we can print topologiq's outputs to inspect them, in particular, `lattice_nodes` and `lattice_edges`, which will become the TQEC `block_graph`.

In [ ]:
if lattice_nodes and lattice_edges:
    print("\nCubes in final output:")
    for k, v in lattice_nodes.items():
        print(f"{k}:{v}")

    print("\nPipes in final output:")
    for k, v in lattice_edges.items():
        print(f"{k}:{v}")

## Importing to TQEC

The structure of `lattice_nodes` and `lattice_edges` is essentially the same as TQEC's `block_graph`, with one caveat.

There is a need to perform some re-scaling and re-indexing of edges. In TQEC's `block_graph`, pipes occupy no space, which leads to adjacent cubes being closer to one another than in a space-time diagram where pipes do occupy space for the purposes of visualising things clearly.

Fortunately, TQEC already has a way to import 3D objects like `lattice_nodes` and `lattice_edges`. TQEC can import 3D models encoded in COLLADA format (`.dae`), which use the same kind of coordinates and artifacts as `lattice_nodes` and `lattice_edges`. Accordingly, the following block of code relies on TQEC's COLLADA interoperability to realise similar transformations for `lattice_nodes` and `lattice_edges`, leading into a `block_graph`.

In [ ]:
if lattice_nodes and lattice_edges:

    from tqec.interop.pyzx.topologiq import read_from_lattice_dicts
    from tqec.computation.block_graph import BlockGraph

    # Import using standardised method for importing lattice surgeries / space-time diagram into TQEC
    lattice_edges_min = dict([(k,v[0]) for k,v in lattice_edges.items()])
    block_graph: BlockGraph = read_from_lattice_dicts(lattice_nodes, lattice_edges_min, graph_name=full_circuit_name)

    # Visualise using TQEC methods
    html = block_graph.view_as_html()
    display(html)

You may have noticed the imported blockgraph lacks the boundary nodes / port cubes, the ones seen in gray in topologiq's 3D visualisation. 

This is normal. The TQEC import adds transparent placeholder objects at those coordinates.

To enable computation, these need to be "filled", which is done as follows.

In [ ]:
if block_graph:

    # Fill ports using function that yields minimal set of block_graphs needed for simulation
    filled_block_graphs = block_graph.fill_ports_for_minimal_simulation()

    # Show all block_graphs in minimal set
    for i, filled_block_graph in enumerate(filled_block_graphs):
        for j, correlation_surface in enumerate(filled_block_graph.observables):
            html = filled_block_graph.graph.view_as_html(
                pop_faces_at_directions=("-Y", "+X"),
                show_correlation_surface=filled_block_graph.observables[j],
            )
            display(html)

## Further usage

We can use `filled_block_graphs` further. For instance, you can use it to get Stim circuits for each basis, as well as run simulations with it. 

Having said that, this will not be covered in this notebook mainly because these steps are documented extensively in other gallery notebooks. 

For guidance on how to generate Stim circuits and run simulations see, for example, [this notebook](https://tqec.github.io/tqec/gallery/qrisp.html).